In [2]:
import json
import os
from tqdm import tqdm
from glob import glob
from streaming import MDSWriter, LocalDataset

columns = {
    'audio_filename': 'str',
    'text': 'str',
}
hashes = 'sha1', 'xxh64'

Define files (add all from provided datasets)

In [3]:
files = [
    '/Users/halimshukor/Documents/github-projects/whisper-distill-finetune/nusantara-data/prepared-nusantara-original.jsonl',
    '/Users/halimshukor/Documents/github-projects/whisper-distill-finetune/malaysian-youtube-data/prepared-pseudolabel.jsonl'
]

In [4]:
!wc -l {files[0]}

    6274 /Users/halimshukor/Documents/github-projects/whisper-distill-finetune/nusantara-data/prepared-nusantara-original.jsonl


In [5]:
!wc -l {files[1]}

 3085595 /Users/halimshukor/Documents/github-projects/whisper-distill-finetune/malaysian-youtube-data/prepared-pseudolabel.jsonl


In [ ]:
with MDSWriter(out='mosaic-whisper', columns=columns, compression=None, hashes=hashes) as out:
    for f in files:
        folder = os.path.split(f)[0]
        with open(f) as fopen:
            for l in tqdm(fopen):
                l = json.loads(l)
                for r in l:
                    audio_filename = os.path.join(folder, r['audio_filename'])
                    audio_filename = os.path.abspath(audio_filename)
                    t = r['text'].strip()
                    before = t
                    t = ' '.join([w for w in t.split() if len(w) > 1]).strip()
                    if len(t) < 3:
                        continue
                    d = {
                        'text': t,
                        'audio_filename': audio_filename,
                    }
                    out.write(d)


Streaming from dataset

In [ ]:
from torch.utils.data import DataLoader
from streaming import StreamingDataset

# Remote path in hugginface
remote = 'hf://your_username/your_dataset_name'
local = '/tmp/path-to-dataset'

dataset = StreamingDataset(local=local, remote=remote, shuffle=True)

row = dataset[0]
text = row['text']
audio = row['audio_filename']

dataloader = DataLoader(dataset)